In [67]:
import tkinter as tk
from tkinter import ttk
from datetime import datetime, timedelta
import requests
from tkinter import messagebox
import pandas as pd
import base64
import os
import json


class ProposicoesGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Consulta de Proposições")
        style = ttk.Style()

    
        input_frame = ttk.Frame(self.root, padding="20")
        input_frame.grid(padx=20, pady=20)
        ttk.Label(input_frame, text="Palavras-chave:").grid(row=0, column=0, sticky="w")
        self.keywords_entry = ttk.Entry(input_frame, width=40)
        self.keywords_entry.grid(row=0, column=1, padx=10, pady=5)
        
        ttk.Label(input_frame, text="Data Início (AAAA-MM-DD):").grid(row=1, column=0, sticky="w")
        self.data_inicio_entry = ttk.Entry(input_frame, width=15)
        self.data_inicio_entry.grid(row=1, column=1, padx=10, pady=5)
        
        
        consultar_button = ttk.Button(input_frame, text="Consultar", command=self.consultar_proposicoes)
        consultar_button.grid(row=3, columnspan=2, pady=10)


        def exibir_selecao():
            valor_checkbox = checkbox_var.get()
            if valor_checkbox == 1:
                style.configure("TextColor.TLabel", foreground="green")
                status_label.config(text="As palavras serão atualizadas na automação", style="TextColor.TLabel")
                self.valorDoCheckbox = 1
            else:
                style.configure("TextColor.TLabel", foreground="red")
                status_label.config(text="As palavras NÃO serão atualizadas na automação 24h", style="TextColor.TLabel")
                self.valorDoCheckbox = 0

        
        checkbox_var = tk.IntVar(value=1)
        checkbox = ttk.Checkbutton(input_frame, text="Salvar palavras chaves", variable=checkbox_var, command=exibir_selecao)
        checkbox.grid(row=5, columnspan=2, pady=10)

        status_label = ttk.Label(input_frame, text="")
        status_label.grid(row=20, columnspan=2, pady=10)
        
        automatizar_button = ttk.Button(input_frame, text="Mostrar palavras salvas", command=self.mostrarPalavrasChaves)
        automatizar_button.grid(row=6, columnspan=2, pady=10) 

        
        self.projetos_lista = []
        self.keys = []
        self.valorDoCheckbox = 1
        

    def mostrarPalavrasChaves(self):
        file_path = 'chaves.bin'

        with open(file_path, 'rb') as f:
            binary_data = f.read()

        readable_data = binary_data.decode('utf-8')
 
        palavras_window = tk.Toplevel(self.root)
        palavras_window.title("Palavras Chaves")
        palavras_window.geometry("400x200")
        palavras_window.configure(bg="#f0f0f0") 
        
        style = ttk.Style()
        style.configure("Custom.TLabel", foreground="#333", font=("Arial", 14, "bold"))
        
        palavras_label = ttk.Label(palavras_window, text=f"Palavras-chave em produção:\n\n{readable_data}", style="Custom.TLabel", background="#f0f0f0")
        palavras_label.pack(padx=20, pady=20, fill=tk.BOTH, expand=True)
        
    
    def consultar_proposicoes(self):
        hoje = pd.Timestamp.today()
        ontem = pd.Timestamp.today() - pd.Timedelta(days=1)
        ontem = ontem.strftime("%Y-%m-%d")
                
        base_url_camara = "https://dadosabertos.camara.leg.br/api/v2"
        base_url_senado = "https://legis.senado.leg.br/dadosabertos"

        endpoint_camara = "/proposicoes"
        endpoint_senado = "/materia/pesquisa/lista"
        
        keywords = self.keywords_entry.get()
        data_inicio = self.data_inicio_entry.get()


        data1 = datetime.strptime(data_inicio, "%Y-%m-%d")
        data2 = hoje
        diferenca = data2 - data1
        
        params_camara = {
            "keywords": keywords,
            "dataInicio": data_inicio,
        }
        
        params_senado = {
            "palavraChave": keywords,
            'ordenacao': 'ASC'
        }

        headers = {
            "Accept": "application/json"
        }
        
        try:
            response_camara = requests.get(base_url_camara + endpoint_camara, params=params_camara, headers=headers)
            if response_camara.status_code == 200:
                dados_camara = response_camara.json()
                proposicoes = dados_camara['dados']
                for proposicao in proposicoes:
                    url_procura_id = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{proposicao['id']}"
                    projeto_id = requests.get(url_procura_id)
                    resposta_projeto = projeto_id.json()
                    projeto_dict_camara = {
                        "ID": resposta_projeto['dados']['id'],
                        "Tipo": resposta_projeto['dados']['siglaTipo'],
                        "Ano": resposta_projeto['dados']['ano'],
                        "Ementa": resposta_projeto['dados']['ementa'],
                        "Data Inicio": resposta_projeto['dados']['dataApresentacao'],
                        "Ultma Alteracao": resposta_projeto['dados']['statusProposicao']['dataHora'],
                        "Palavras Chaves": resposta_projeto['dados']['keywords'],
                        "Link": f"https://www.camara.leg.br/proposicoesWeb/fichadetramitacao?idProposicao={resposta_projeto['dados']['id']}"
                    }
                    self.projetos_lista.append(projeto_dict_camara)
            else:
                messagebox.showerror("Erro", f"Código de status da requisição: {response_camara.status_code}")
                
            response_senado = requests.get(base_url_senado + endpoint_senado, params=params_senado, headers=headers)
            if response_senado.status_code == 200:
                dados_senado = response_senado.json()
                if ('Materias' in dados_senado['PesquisaBasicaMateria']):
                    materias = dados_senado['PesquisaBasicaMateria']['Materias']
                    for proposicao in materias['Materia']:
                        url_procura_id_senado_atualizacao = f"https://legis.senado.leg.br/dadosabertos/materia/atualizacoes/{proposicao['Codigo']}?numdias={diferenca.days+1}"
                        projeto_id_senado = requests.get(url_procura_id_senado_atualizacao, headers=headers)
                        resposta_projeto_senado_atualizao = projeto_id_senado.json()

                        url_procura_id_senado = f"https://legis.senado.leg.br/dadosabertos/materia/{proposicao['Codigo']}"
                        projeto_id_senado = requests.get(url_procura_id_senado, headers=headers)
                        resposta_projeto_senado = projeto_id_senado.json()
                        

                  
                        projeto_dict_senado = {
                            "ID": proposicao['Codigo'],
                            "Tipo": proposicao['Sigla'],
                            "Ano": proposicao["Ano"],
                            "Ementa": proposicao["Ementa"],
                            "Data Inicio": proposicao['Data'],
                            "Link": f"https://www25.senado.leg.br/web/atividade/materias/-/materia/{proposicao['Codigo']}"
                        }
                            
                        if ('Materia' in resposta_projeto_senado_atualizao['AtualizacaoMateria']):
                            projeto_dict_senado["Ultma Alteracao"] = resposta_projeto_senado_atualizao['AtualizacaoMateria']["Materia"]["AtualizacoesRecentes"]["Atualizacao"][0]["DataUltimaAtualizacao"]
                        else:
                            continue
                        if('Classificacoes' in resposta_projeto_senado['DetalheMateria']["Materia"].keys()):
                            projeto_dict_senado["Palavras Chaves"] = resposta_projeto_senado['DetalheMateria']["Materia"]['Classificacoes']['Classificacao'][0]['DescricaoClasse']
                            self.projetos_lista.append(projeto_dict_senado)
                            continue
                                
                        elif('IndexacaoMateria' in resposta_projeto_senado['DetalheMateria']["Materia"]['DadosBasicosMateria'].keys()):
                            projeto_dict_senado["Palavras Chaves"] = resposta_projeto_senado['DetalheMateria']["Materia"]['DadosBasicosMateria']['IndexacaoMateria']
                            self.projetos_lista.append(projeto_dict_senado)

                relatorio_df = pd.DataFrame(self.projetos_lista)
                relatorio_df.to_excel("Projetos-Novos.xlsx", index=False)
                messagebox.showinfo("Sucesso", "Consulta realizada com sucesso. Resultados salvos em Projetos-Novos.xlsx.")

                os.startfile("Projetos-Novos.xlsx")
            
                salvar_chaves = self.valorDoCheckbox
                if salvar_chaves == 1:
                    keys = self.keywords_entry.get()
                    self.keys.append(keys)
    
                    dado = f"{keys}"
                    
                    dado = ', '.join(self.keys)
                    dado = dado.encode()
                    
                    file_path = 'chaves.bin'
                    
                    try:
                        if os.path.exists(file_path):
                            os.remove(file_path)
                            print(f"Arquivo {file_path} removido com sucesso.")
                        else:
                            print(f"Arquivo {file_path} não encontrado.")
                    except Exception as e:
                            print(f"Ocorreu um erro ao tentar remover o arquivo {file_path}: {str(e)}")
        
                    with open(file_path, 'wb') as f:
                        f.write(dado)
                        
                        print(f'Arquivo binário "{file_path}" criado e os dados foram escritos.')
                        self.keys.clear()
                        keywords = None
                    self.root.destroy()
                    self.root.quit()
            else:
                messagebox.showerror("Erro", f"Código de status da requisição: {response_senado.status_code}")
        
        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro ao realizar a consulta: {str(e)}")

def main():
    root = tk.Tk()
    app = ProposicoesGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Arquivo chaves.bin removido com sucesso.
Arquivo binário "chaves.bin" criado e os dados foram escritos.
